In [37]:
# 数字的取值为 0~1024，那么最多可能性为 2047种，即 bin(1024)每一位都是1, bin(1024) = 10000000000

In [36]:
print(1024+512+256+128+64+32+16+8+4+2+1)
print(bin(1024))

2047
0b10000000000


In [67]:
from typing import List
from collections import defaultdict

class Solution:
    def minChanges(self, nums: List[int], k: int) -> int:
        
        def calcCost(idx, val):
            # 将下标为 idx、idx+k、idx+2k ...的元素修改为val，需要更改多少数字
            sum_val = total[idx]
            return sum_val - count[idx][val]
        
        # 统计每个位置上，数字出现的频次，和每个位置元素的总数
        count = [defaultdict(int) for _ in range(k)]
        total = [0] * k
        for i, v in enumerate(nums):
            count[i % k][v] += 1
            total[i % k] += 1
     
        # 定义：dp[i][j] 表示 nums中的前 i 个元素的异或值为 j最小操作次数
        n = len(nums)
        dp = [[float('inf')] * 2048 for _ in range(k)]
        for i in range(1024):
            dp[0][i] = calcCost(0, i)
       
        for i in range(1, k):
            
            pre_min_cost = float('inf')
            pre_d = None
            for d in range(1024):
                if dp[i-1][d] < pre_min_cost:
                    pre_min_cost = dp[i-1][d]
                    pre_d = d
            
            for d in range(1024):
                # 前 i 个数字的异或和为 v，前一轮的 异或和 取值为 pre_d, 
                # 那么就需要把 i-th 数字更改为 pre_d ^ v = v
                dp[i][d] = pre_min_cost + calcCost(i, d ^ pre_d)
            
                for v in count[i]: # v2是 i、i+k、i+2k....位置上的数字
                    dp[i][d] = min(dp[i][d], dp[i-1][v^d] + calcCost(i, v))
        return dp[-1][0]

In [68]:
solution = Solution()
solution.minChanges([26,19,19,
                     28,13,14,
                     6,25,28,
                     19,0,15,
                     25,11], 3)

11

In [90]:
from typing import List
from collections import Counter, defaultdict

class Solution:
    # 思路：
    #   可知所求为形如： A[i] = A[i + k], 其中 A[i] -> A[i + k - 1] 的异或和 0
    #   即所求，将形成 n // k 个分组，一个组的大小为 k. 而这个组中所有的数字异或和为 0.

    #   将 nums 数组拆分为 k 个 group, 并统计这个 group 中每个数字出现的 count。
    #   动态规划思想： dp[state] 表示当前组，所得到 state 的异或值，的最小操作次数。
    #                当完成所有的 group 的计算时，最后的 dp[0] 即为所求。

    def minChanges(self, nums: List[int], k: int) -> int:
        n = len(nums)
        count = [Counter([nums[i] for i in range(j, n, k)]) for j in range(k)]
        
        # case 1: one number is not from list
        # 每个位置最多出现重复的数字有多少次
        mxs = [count[i].most_common(1)[0][1] for i in range(k)]
        
        ans = n - (sum(mxs) - min(mxs))
        # key:异或值，val：达到此异或值不需要改动数字的最大数量
        dp = {0: 0} # 异或值为 0，不需要改动的数字的数量为多少，初始化为 0 
        
        for i in range(k):
            new_dp = defaultdict(int)
            cur = count[i]
            # 当前位置各个数字出现的频次
            for k, v in cur.items():
                for s in dp: # 前一轮的异或值有哪些可能性
                    new_dp[k ^ s] = max(new_dp[k ^ s], dp[s] + v)
            dp = new_dp
        
        print(dp[0])
        return min(ans, n - dp[0])

In [91]:
solution = Solution()
solution.minChanges([1,2,4,1,2,5,1,2,6], 3)

Counter({1: 3}) [(1, 3)]
0


3

In [ ]:
dp[i][v] = dp[i-1][v^d] + calcCost(i % k, v)
# 前 i 个元素的异或和为 d，那么前 i-1个数字的异或和就是 v^d, 
# cost：将第i号位置的所有元素都更改为v的代价